# Cvičenie 11: Hry v Pythone

Na našom poslednom cvičení v tomto semestri si ukážeme, ako môžete vytvoriť jednoduché hry v Pythone. Síce na tieto účely existuje niekoľko knižníc, my na cvičení budeme používať `pygame`, ktorá ponúka celú základnú funkcionalitu pre vykresľovanie objektov, posielanie správ a prehrávanie hudby. Okrem toho rieši aj štandardné problémy, ako napríklad dotyk objektov. Ak počas vypracovania úlohy budete mať otázky alebo neistoty, môžete si prečítať viac o použitých funkciách [v dokumentácii knižnice `pygame`](https://www.pygame.org/docs/).

Na úvod si však potrebujete nainštalovať knižnicu, napríklad cez príkazový riadok (alebo v Anaconde):

```
pip install pygame
```

Následne si stiahnite [predpripravený projekt s riešením](sources/lab11/lab11.zip).

Dnešné cvičenie bolo vypracované na základe tutorialu [dostupného tu](https://coderslegacy.com/python/python-pygame-tutorial/). Na tejto stránke nájdete [aj ukážku hry, ktorú si vytvoríme](https://coderslegacy.com/wp-content/uploads/2020/06/Pygame_Tutorial.mp4).

## 1. Oboznámime sa s kódom

V pripravenom skripte už nájdete kostru riešenia s niekoľkými triedami aj základnou funkcionalitou. Na úvod si naštudujeme, čo tieto riadky robia a ako nám pomôžu pri riešení.

Na začiatku tu hneď máme importy, kde pracujeme aj so štandardnými modulmi, aj s knižnicou `pygame`. Tieto importy využijeme nasledovne:

* `random` - pre generovanie náhodnej štartovacej pozície protivníka;
* `sys` - pre správne ukončenie aplikácie a uzavretie nášho okna;
* `time` - pre zastavenie aplikácie, kým sa prehrá ukončovacia hudba.

In [ ]:
import random
import sys
import time

import pygame
from pygame.locals import *

V ďalšom bloku sa vykoná hneď niekoľko dôležitých krokov, najprv si nainicializujeme `pygame` hru pomocou volania funkcie `init`, ktorá nám pripraví prostredie pre spúšťanie hry. Je to neoddeliteľnou súčasťou každej `pygame` aplikácie.

In [ ]:
pygame.init()

V ďalšom kroku sa inicializuje frekvencia vykresľovania hry na obrazovku pomocou hodnoty `FPS` (*frames per second*), teda koľko snímok sa vykreslí na obrazovku za sekundu. Pre tieto účely vytvoríme aj `pygame` reprezentáciu tejto aktualizácie, aby výsledná hra bola plynulá a nevyzerala tak, ako keby sekala.

In [ ]:
FPS = 60
FramePerSec = pygame.time.Clock()

Ďalej riešime niekoľko grafických záležitostí. Najprv si zadefinujeme niekoľko farieb, ktoré neskôr využijeme. Tieto farby sú definované ako trojice udávajúce jas po kanáloch **R**ed, **G**reen, **B**lue. Následne ešte definujeme veľkosť obrazovky cez konštanty `SCREEN_WIDTH` a `SCREEN_HEIGHT`. `SPEED` bude reprezentovať rýchlosť nášho auta (resp. protivníkov), a `SCORE` budeme aktualizovať pre uchovávanie aktuálneho skóre v hre.

In [ ]:
RED = (255, 0, 0)
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

SCREEN_WIDTH = 400
SCREEN_HEIGHT = 600
SPEED = 5
SCORE = 0

Ešte s grafikou súvisí vypisovanie informácií na obrazovku. K tomu v `pygame` si potrebujete vytvoriť najprv objekty fontov, ktoré sa použijú na prípravu textov. Neskôr si vygenerujete texty pomocou nich (cez metódu `render`), a môžete vykresliť až takto vygenerovaný objekt. V našej hre budeme používať dva fonty, prvý (väčšie písmo) sa použije pri výpise pri prehratí hry, a druhý (menšie písmo) budeme používať na vypisovanie skóre. Keďže ukončovaciu správu vypíšeme iba raz, hneď teraz si pripravíme objekt, ktorý sa neskôr vypíše.

In [ ]:
font = pygame.font.SysFont("Verdana", 60)
game_over = font.render("Game Over", True, BLACK)
font_small = pygame.font.SysFont("Verdana", 20)

Posledná vec, ktorú urobíme v prípravnej fáze súvisí s plátnom, na ktoré budeme vykresľovať hru. K tomu si najprv vytvoríme `DISPLAYSURF`, ktorá v sebe bude uchovávať referenciu na obrazovku, ktorej nastavíme veľkosť podľa konštánt. Našu obrazovku zafarbíme bielou farbou, a nášmu oknu nastavíme ľubovoľný názov.

Okrem toho sa tu ešte načíta obrázok, ktorý neskôr použijeme ako pozadie hry.

In [ ]:
background = pygame.image.load("resources/AnimatedStreet.png")

DISPLAYSURF = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
DISPLAYSURF.fill(WHITE)
pygame.display.set_caption("Game")

Ďalej kód obsahuje definíciu dvoch tried, ku ktorým sa vrátime neskôr. Teraz dôležitejší je posledný blok kódu, ktorý rieši základy hernej logiky. Ako môžete vidieť, najprv si zadefinujeme dva objekty - jedného hráča, jedného protivníka. Následne sa spustí herná logika ako nekonečný cyklus, ktorý sa ukončí až na základe podmienky.

Podmienka skontroluje, či sa medzi eventmi vygenerovanými počas hry nenachádza event typu `QUIT`, teda udalosť určujúca potrebu ukončenia hry. Eventy sú objekty reprezentujúce ľubovoľnú udalosť, alebo aj správu, a používajú sa veľmi často v grafických aplikáciách ale aj vo viacvláknových riešeniach. Ak potrebný event sa nájde, tak hru uzavrieme (`pygame.quit()`) a ukončíme aj našu aplikáciu (`sys.exit()`).

Následne budeme aktualizovať naše objekty cez príslušné metódy (implementácia sa doplní neskôr). Ako môžete vidieť, na ďalšom riadku už znova zafarbíme plátno na bielu. Dôvodom toho je, že rôzne vrstvy sa vykresľujú v takom poradí, ako zavoláme príslušné metódy. Keďže my budeme pridávať aj autá, tak bez opätovného zafarbenia plátna tieto autá by sa stali súčasťou scény aj pri ďalšom kroku vykresľovania, ako aj pri treťom, štvrtom, atď. kole. Dostali by sme teda obrazovku, na ktorej by sme videli okrem súčasnej pozície aj všetky predchádzajúce pozície áut. Ukážkové vykresľovanie auta vidíte ako zakomentovaný riadok, keďže zatiaľ nezadefinovali sme všetky potrebné atribúty triedy.

Na konci cyklu sa ešte aktualizuje obrazovka (`pygame.display.update()`), a zavolá sa metóda pre simuláciu času tak, aby bola dodržaná zadefinovaná frekvencia (`FPS`).

In [ ]:
P1 = Player()
E1 = Enemy()


while True:
    for event in pygame.event.get():
        if event.type == QUIT:
            pygame.quit()
            sys.exit()
    P1.move()
    E1.move()

    DISPLAYSURF.fill(WHITE)
    # DISPLAYSURF.blit(P1.image, P1.rect)

    pygame.display.update()
    FramePerSec.tick(FPS)


## 2. Definícia hráča

V ďalsom kroku zadefinujeme triedu reprezentujúcu hráča, a pridáme aj ovládanie.

**Úloha:** Najprv rozšírte konštruktor triedy `Player` tak, aby sa načítal príslušný obrázok z priečinka `resources`. Ako príklad si vezmite načítanie obrázku pozadia. V konštruktore ešte nastavte členskú premennú `rect` ako obdĺžnik obrázka, ktorý získate príslušnou metódou z triedy [`pygame.image`](https://www.pygame.org/docs/ref/image.html), prípadne jej nadtriedy [`pygame.Surface`](https://www.pygame.org/docs/ref/surface.html). Centrum obdĺžnika (atribút `self.rect.center`) nastavte tak, aby sa hráč nachádzal na dolnej časti obrazovky niekde v strede (nezabudnite, že v počítačovej grafike za bod `[0, 0]` sa berie horný ľavý roh).

**Úloha:** Implementujte metódu `move`, ktorá rieši pohyb hráča. Najprv získajte zoznam všetkých stlačených kláves cez funkciu [`pygame.key.get_pressed()`](https://www.pygame.org/docs/ref/key.html#pygame.key.get_pressed), a následne skontrolujte, či boli stlačené šípky doľava resp. doprava. [Zoznam kódov kláves nájdete tu.](https://www.pygame.org/docs/ref/key.html). Pre plynulosť pohybu auto posuňte nie o jednu pozíciu, ale napr. o 5 po x-ovej osi (aktualizujte polohu objektu [`rect`](https://www.pygame.org/docs/ref/rect.html)). Ošetrite, aby hráč neopustil herné pole.

In [ ]:
class Player(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()

    def move(self):
        pass

Svoje riešenie môžete otestovať odkomentovaním príslušného riadku v hlavnej slučke hry a opätovným spustením skriptu.

## 3. Definícia protivníka

Aby hra bola zaujímavá, potrebujeme ešte zadefinovať aj protivníka, teda auto, ktorému sa hráč bude snažiť vyhnúť. Trieda `Enemy` pričom má podobnú funkcionalitu ako trieda `Player` na zopár rozdielov:

* v konštruktore si načítajte príslušný obrázok, získajte z neho obdĺžnik, a centrum obdĺžnika dajte na náhodné miesto po x-ovej osi, a na 0 po y-ovej osi;
* v metóde `move` auto posúvajte stále dole po y-ovej osi o hodnotu `SPEED` (neskôr sa bude zvyšovať). Ak auto vyjde z hracej plochy (vrch obdĺžnika bude už mimo), tak ho vráťte na začiatok obrazovky podobne ako v konštruktore nastavením na náhodné miesto po hornom okraji.

In [ ]:
class Enemy(pygame.sprite.Sprite):
    def __init__(self):
        super().__init__()

    def move(self):
        pass

## 4. Skupiny a vykresľovanie postáv

Namiesto toho, aby sme hráča aj protivníka vykresľovali manuálne pridávaním príslušných riadkov (čo by bolo dosť nemožné pri dynamickom generovaní týchto postáv), vytvoríme si skupiny postáv, teda sprite-ov, ktoré slúžia na grafickú reprezentáciu našich áut. Použijeme na to triedu [`Group`](https://www.pygame.org/docs/ref/sprite.html#pygame.sprite.Group) z modulu `pygame.sprite` ešte pred samotným spustením hernej logiky:

In [ ]:
P1 = Player()
E1 = Enemy()

enemies = pygame.sprite.Group()
# TODO: add enemy to the group
all_sprites = pygame.sprite.Group()
# TODO: add enemy and player to the group

Následne v hernej slučke vykresľovanie vyriešte tak, aby ste prechádzali zoznamom všetkých sprite-ov, a následne vykreslili každú postavu na svoju pozíciu (môžete použiť rovnaký prístup ako pred tým pre hráča, len to zovšeobecnite). Pri prechádzaní zoznamom naďalej zavolajte aj metódu `move`, aby sa naši postavy pohybovali.

Upravte aj vykresľovanie samotnej hernej plochy - namiesto bieleho pozadia tam chceme mať pozadie, ktoré sme si načítali do premennej `background` (vykreslite ju na pozíciu `[0, 0]`).

## 5. Ukončenie hry

Ak ste postupovali správne, tak naše postavy sa už hýbu, t.j. dokážeme ovládať nášho hráča, a protivník pekne ide v protismere. Ak sa tieto dve autá náhodou stretnú, nič sa neudeje, hra pokračuje ďalej, ako keby sa nič nestalo.

K tomu, aby sme vedeli identifikovať, či sa hra skončila (došlo k zrážke), potrebujeme začať riešiť kolízie. Pridajte túto funkcionalitu do hlavnej hernej slučky, inšpirujte sa metódami z modulu [`pygame.sprite`](https://www.pygame.org/docs/ref/sprite.html#pygame.sprite).

Ak dôjde ku kolízii, urobte nasledovné zmeny:

* nastavte pozadie obrazovky na červenú, a vypíšte správu *GAME OVER* (objekt bol vytvorený na začiatku, vypisuje sa rovnako ako ľubovoľný sprite);
* aktualizujte obrazovku;
* "zabite" (*kill*nite) každý sprite - metóda [`kill`](https://www.pygame.org/docs/ref/sprite.html#pygame.sprite.Sprite.kill) odstráni sprite z každej skupiny, a takto predídeme prípadnej aktualizácii sprite-ov;
* počkajte niekoľko sekúnd (funkcia zo štandardného modulu [`time`](https://docs.python.org/3/library/time.html)) a následne ukončite hru podobne ako v prípade **QUIT** eventu.

## 6. Obťažnosť

Teraz už správne identifikujeme kolízie, vzhľadom ale na rýchlosť protiidúcich áut je dosť nepravdepodobné, že ku zrážke dôjde. Práve preto v tomto kroku pridáme stále vyššiu obťažnosť do hry, a to tak, že časom zrýchlime protiidúce autá. Použijeme pritom aktualizáciu premennej `SPEED` (ktorá sa používa pri presúvaní protivníka) a [eventy](https://www.pygame.org/docs/ref/event.html).

Eventy sú objekty, ktoré reprezentujú ľubovoľnú udalosť. Síce existujú štandardné eventy (ako napríklad **QUIT** pre vypnutie hry), vieme si takisto zadefinovať aj vlastné pomocou konštanty `pygame.USEREVENT`, napríklad:

```
INC_SPEED = pygame.USEREVENT + 1
```

nám zadefinuje nový typ eventu `INC_SPEED` s vlastným kódom. Tento event však potrebujeme vytvárať, aby sme ho mohli odchytiť.

K tomu máme niekoľko možností, v našom riešení však teraz použijeme časomieru, teda rýchlosť budeme zvyšovať po každej sekunde. V module [`pygame.time`](https://www.pygame.org/docs/ref/time.html) si nájdite metódu, ktorá vygeneruje nám event daného typu po istom čase, a zadefinujte vlastný typ aj časomieru ešte pred hlavnou slučkou hry. Následne hlavnú slučku rozšírte tak, aby pri eventoch sa skontroloval aj výskyt eventu `INC_SPEED` a nie len `QUIT`. V takomto prípade inkrementujte hodnotu `SPEED` podľa vlastného návrhu.

Po opätovnom spustení by sa protihráči mali pohybovať stále rýchlejšie a rýchlejšie.

## 7. Skóre

Naša hra už celkom dobre funguje, avšak bolo by dobré, keby sme počet áut, ktorým sme sa vyhli nemuseli počítať my (pri veľkej rýchlosti protivníkov sa to ani nedá). Práve preto zavedieme a vypíšeme skóre, pričom potrebujete urobiť nasledovné zmeny:

* v metóde `Enemy.move` pridajte aktualizáciu počítadla `SCORE`, ak sa protiidúce auto dostane mimo plochy (vyhli sme sa mu); k tomu potrebujete premennú `SCORE` urobiť globálnou príkazom `global SCORE` na začiatku metódy;
* vypíšte aktuálne skóre v hlavnej slučke hry (postupujte podobne ako pri správe *GAME OVER*), použite k tomu font `font_small`. Nezabudnite, že najprv musíte vyrenderovať objekt, a ten následne vykresliť na vami zvolené miesto (tak, aby text neprekážal, napr. `[10, 10]`).

## 8. It All Ended with a Big Bang! (Bang!)

Našou poslednou úlohou je pridať prehranie zvuku, aby sme hráčovi dali vedieť, že prehral. Pre audiostopu budeme používať triedu [`pygame.mixer.Sound`](https://www.pygame.org/docs/ref/mixer.html#pygame.mixer.Sound), ktorá definuje metódu pre prehratie tohto zvuku. Metódu zavolajte pri zrážke. Pre lepšie fungovanie odporúčame hneď po prehratí zvuku vykonávanie programu pozastaviť zavolaním metódy `time.sleep` približne na pol sekundy.

Tým vývoj hry nateraz ukončíme, ale samozrejme nič vám nebráni v tom, aby ste ďalej experimentovali touto hrou a knižnicou `pygame`.